In [1]:
#! -*- coding:utf-8 -*-
#code write by guohao
 
import itertools
 
class Apriori(object):
    
    def __init__(self,min_sup=0.2,dataDic={}):
        self.data = dataDic      #构建数据记录词典
        self.size = len(dataDic) #统计事务个数
        self.min_sup = min_sup   #最小支持度阈值
        self.min_sup_val = min_sup * self.size  #最小支持度计数
 
    def find_frequent_1_itemsets(self):
        FreqDic = {}   #用于统计物品的支持度计数
        for event in self.data:    #event为每一条记录
            for item in self.data[event]:   #item为项
                if item in FreqDic:
                    FreqDic[item] += 1
                else:
                    FreqDic[item] = 1
 
        L1 = []
        for itemset in FreqDic:
            if FreqDic[itemset] >=self.min_sup_val:  #过滤掉小于最小支持度计数的1-项集
                L1.append([itemset])
        return L1
 
    def has_infrequent_subset(self,c,L_last,k):
        #c为当前集合，L_last为上一个频繁项集的集合，k为当前频繁项集内的元素个数
        #该函数用于检查当前子集是否都为频繁项集
        subsets = list(itertools.combinations(c,k-1))
        for each in subsets:
            each = list(each)
            if each not in L_last:
                return True
        return False
 
    def apriori_gen(self,L_last):  #连接步实现
        k = len(L_last[0]) + 1
        Ck = []
        for itemset1 in L_last:
            for itemset2 in L_last:
                flag = 0
                for i in range(k-2):
                    if itemset1[i] != itemset2[i]:    #如果前k-2项中有一项不相等，则合并的项集必定不为频繁项集
                        flag = 1
                        break
                if flag == 1:continue
                if itemset1[k-2] < itemset2[k-2]:
                    c = itemset1 + [itemset2[k-2]]    #合成待定k项集
                else:
                    continue
 
                if self.has_infrequent_subset(c,L_last,k):  #判断是否为1-项集
                    continue
                else:
                    Ck.append(c)
        return Ck
 
    def do(self):
        L_last = self.find_frequent_1_itemsets()   #找到频繁一项集
        L = L_last
        i = 0
        while L_last != []:
            Ck = self.apriori_gen(L_last)      #合并形成新的待定频繁项集
            FreqDic = {}         #项集的频数集合
            for event in self.data:
                for c in Ck:    
                    if set(c) <= set(self.data[event]): #判断新合成的频繁集是否是事务记录的子集
                        if tuple(c) in FreqDic:
                            FreqDic[tuple(c)] += 1
                        else:
                            FreqDic[tuple(c)] = 1
            Lk = []
            for c in FreqDic:
                if FreqDic[c] > self.min_sup_val: #判断新形成的待定频繁项集是否为为真正的频繁项集
                    Lk.append(list(c))
            L_last = Lk
            L += Lk
        return L
 
 


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from string import punctuation
from sklearn import svm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from nltk import ngrams
from itertools import chain
from wordcloud import WordCloud

#数据导入
file_path = './Problem_C_Data/hair_dryer.tsv'
df = pd.read_csv(file_path, sep='\t', header=0)
df.head()

df['Helpful %'] = np.where(df['helpful_votes'] > 0, df['helpful_votes'] / df['total_votes'], -1)
df['% Upvote'] = pd.cut(df['Helpful %'], bins = [-1, 0, 0.2, 0.4, 0.6, 0.8, 1.0], labels = ['Empty', '0-20%', '20-40%', '40-60%', '60-80%', '80-100%'], include_lowest = True)
df.head()

df_s = df.groupby(['star_rating', '% Upvote']).agg({'review_id': 'count'})
df_s = df_s.unstack()
df_s.columns = df_s.columns.get_level_values(1)
fig = plt.figure(figsize=(15,10))

#保留不是3的评论
filtered_data = df[df['star_rating'] != 3]
def partition(x):
    if x>3:
        return 'positive'
    return 'negative'

actual_score = filtered_data['star_rating']
positiveNegative = actual_score.map(partition)
filtered_data['Score'] = positiveNegative

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [3]:
import re

# Sorting data points according to the 'ProductId'
sorted_data = filtered_data.sort_values('product_id', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

# Eliminating the duplicate data points based on: 'UserId', 'ProfileName', 'Time', 'Summary'
final = sorted_data.drop_duplicates(subset={'review_id', 'product_title', 'review_date', 'review_body'}, keep='first', inplace=False)

# Eliminating the row where 'HelpfulnessDenominator' is greater than 'HelpfulnessNumerator' as these are the wrong entry
final = final[final['total_votes'] >= final['helpful_votes']]

# Getting shape of final data frame
display(final.shape)


# Creating the set of stopwords
stop = set(stopwords.words('english'))

# For stemming purpose
snow = nltk.stem.SnowballStemmer('english')

# Defining function to clean html tags
def cleanhtml(sentence):
    cleaner = re.compile('<.*>')
    cleantext = re.sub(cleaner, ' ', sentence)
    return cleantext

# Defining function to remove special symbols
def cleanpunc(sentence):
    cleaned = re.sub(r'[?|.|!|*|@|#|\'|"|,|)|(|\|/]', r'', sentence)
    return cleaned


# Important steps to clean the text data. Please trace it out carefully
i = 0
str1 = ''
all_positive_words = []
all_negative_words = []
final_string = []
s=''
for sent in final['review_body'].values:
    filtered_sentence = []
    sent = cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if ((cleaned_words.isalpha()) & (len(cleaned_words)>2)):
                if (cleaned_words.lower() not in stop):
                    s = (snow.stem(cleaned_words.lower())).encode('utf-8')
                    filtered_sentence.append(s)
                    if (final['Score'].values)[i] == 'positive':
                        all_positive_words.append(s)
                    if (final['Score'].values)[i] == 'negative':
                        all_negative_words.append(s)
                else:
                    continue
            else:
                continue
    str1 = b" ".join(filtered_sentence)
    final_string.append(str1)
    i += 1
    
# Adding new column into dataframe to store cleaned text
final['CleanedText'] = final_string
final['CleanedText'] = final['CleanedText'].str.decode('utf-8')


# Getting shape of new datset
print(final.shape)

(10471, 18)

(10471, 19)


In [4]:
X = final['CleanedText']
y_dict = {1:0, 2:0, 4:1, 5:1}
y = final['star_rating'].map(y_dict)

from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)
import matplotlib as mpl


mpl.rcParams['font.size']=12                #10 
mpl.rcParams['savefig.dpi']=100             #72 
mpl.rcParams['figure.subplot.bottom']=.1 


def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stopwords,
        max_words=300,
        max_font_size=40, 
        scale=3,
        random_state=1 # chosen at random by flipping a coin; it was heads
        
    ).generate(str(data))
    
    fig = plt.figure(1, figsize=(15, 15))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()
c = CountVectorizer(stop_words = 'english')

def text_fit(X, y, model,clf_model,coef_show=1):
    
    X_c = model.fit_transform(X)
    print('# features: {}'.format(X_c.shape[1]))
    X_train, X_test, y_train, y_test = train_test_split(X_c, y, random_state=0)
    print('# train records: {}'.format(X_train.shape[0]))
    print('# test records: {}'.format(X_test.shape[0]))
    clf = clf_model.fit(X_train, y_train)
    acc = clf.score(X_test, y_test)
    print ('Model Accuracy: {}'.format(acc))

    if coef_show == 1: 
        w = model.get_feature_names()
        coef = clf.coef_.tolist()[0]
        coeff_df = pd.DataFrame({'Word' : w, 'Coefficient' : coef})
        coeff_df = coeff_df.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
        print('')
        print('-Top 20 positive-')
        print(coeff_df.head(20).to_string(index=False))
        print('')
        print('-Top 20 negative-')        
        print(coeff_df.tail(20).to_string(index=False))
    return coeff_df
    
    
coeff_df = text_fit(X, y, c, LogisticRegression())

# features: 7860
# train records: 7853
# test records: 2618
Model Accuracy: 0.9297173414820473

-Top 20 positive-
      Word  Coefficient
      love     2.020178
     excel     1.967720
   perfect     1.775876
     great     1.734631
 complaint     1.727514
   definit     1.698679
    awesom     1.633642
      best     1.612823
      tire     1.525653
      easi     1.464901
      nice     1.430808
      amaz     1.403347
     final     1.308430
       bit     1.303979
      fast     1.259788
      glad     1.256158
   compact     1.234372
   quieter     1.218967
     hesit     1.210404
     fabul     1.209890

-Top 20 negative-
       Word  Coefficient
    special    -1.350388
   descript    -1.365947
    useless    -1.390985
      stuck    -1.415986
      worst    -1.464123
       slow    -1.465845
      smoke    -1.528165
   arthriti    -1.543483
      broke    -1.567801
       poor    -1.595778
     defect    -1.612045
      spark    -1.617485
      month    -1.646086
       fail  

C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [5]:
final.head()


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,Helpful %,% Upvote,Score,CleanedText
10544,US,42867423,R3ATOPWDJHVDMG,B000052YD1,548212381,"Colgate Peroxyl, Mouth Sore Rinse (Hydrogen Pe...",Beauty,5,3,6,N,Y,Peroxyl,Peroxyl isn't available where I live. The ship...,3/11/2010,0.500000,40-60%,positive,peroxyl isnt avail live shipment slower expect...
10309,US,49919466,R1UW4UPNZR9SWU,B00005351F,138665286,"OneTouch Penlet Plus, Adjustable Blood Sampler...",Beauty,4,2,2,N,Y,Great!,My last penlet died pretty quickly. This one s...,8/5/2010,1.000000,80-100%,positive,last penlet die pretti quick one seem lot soli...
10492,US,19498472,R3VSUTOD2WZC0Y,B00005JG0H,801741616,"Neutrogena MoistureShine Gloss, Whisper 05, 0....",Beauty,5,0,0,N,Y,Always try Amazon 1st,"Product is a nice beigey gloss,cannot find it ...",5/7/2010,-1.000000,Empty,positive,product nice beigey glosscannot find store any...
11287,US,53095860,R172BY1R9FMJV4,B00005JS5G,297404894,Panasonic ER389K Rechargeable Beard and Mustac...,Beauty,2,3,3,N,N,My previous ER389 lasted 2 years,I just ordered a new ER389 because my current ...,4/15/2007,1.000000,80-100%,negative,order new current one die littl two year use n...
11468,US,38947355,R3JMGN42OJCL97,B00005O0MZ,694290590,conair corp pers care 146np conair ionic condi...,Beauty,5,78,87,N,N,Excellent for fine/limp hair,This hairdryer far exceeded my expection for<B...,4/20/2002,0.896552,80-100%,positive,hairdryer far exceed expect high recommend fin...


In [6]:
df = final[['review_id','star_rating','CleanedText']]
df.head()

,review_id,star_rating,CleanedText
10544,R3ATOPWDJHVDMG,5,peroxyl isnt avail live shipment slower expect...
10309,R1UW4UPNZR9SWU,4,last penlet die pretti quick one seem lot soli...
10492,R3VSUTOD2WZC0Y,5,product nice beigey glosscannot find store any...
11287,R172BY1R9FMJV4,2,order new current one die littl two year use n...
11468,R3JMGN42OJCL97,5,hairdryer far exceed expect high recommend fin...


In [7]:
from nltk.tokenize import sent_tokenize, word_tokenize
def text_spilt(total_text):
    total_text = total_text.lower()
    #stop = set(stopwords.words('english'))
    total_text = nltk.word_tokenize(total_text)
    lemmatizer = WordNetLemmatizer()
    total_text = [lemmatizer.lemmatize(w,'a') for w in total_text]
    #total_text = [word for word in total_text if word not in stop and len(word) >= 3]
    total_text = [word for word, form in nltk.pos_tag(total_text) if form == 'JJ']
    return total_text

EXAMPLE_TEXT = "This dries my hair faster that bigger, more powerful models. I love travel blow dryers because they are easy to lift and they usually come in 1600 w or less. Bigger dryers are heavy and blow my hair everywhere. This has a surprising amount of power and is very compact. I would give it a five except that the switch is not easy to turn on and off with one hand and it's noisier than I anticipated."
text_spilt(EXAMPLE_TEXT)

['big',
 'powerful',
 'i',
 'easy',
 'big',
 'heavy',
 'blow',
 'surprising',
 'compact',
 'easy',
 'noisy']

In [8]:
df['Text'] = df.apply(lambda x:text_spilt(x['CleanedText']),axis=1)
df = df[['review_id','star_rating','Text']]
df.head()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,review_id,star_rating,Text
10544,R3ATOPWDJHVDMG,5,"[isnt, live, slow, arriv, frame]"
10309,R1UW4UPNZR9SWU,4,"[last, pretti, quick, solid, great, last, long..."
10492,R3VSUTOD2WZC0Y,5,[nice]
11287,R172BY1R9FMJV4,2,"[new, current, nice, last, realli, fill, dead,..."
11468,R3JMGN42OJCL97,5,"[high, fine]"


In [9]:
df1 = df[df['star_rating']==5][['review_id','Text']]
df1 = df1.set_index('review_id').T.to_dict('records')
df1 = df1[0]
df1

{'R3ATOPWDJHVDMG': ['isnt', 'live', 'slow', 'arriv', 'frame'],
 'R3VSUTOD2WZC0Y': ['nice'],
 'R3JMGN42OJCL97': ['high', 'fine'],
 'R2WWTC65M37NZI': ['good'],
 'R11EDRQ0U1MAKG': ['thick',
  'dry',
  'frizz',
  'frizzi',
  'easy',
  'use',
  'overal'],
 'R25XUTMLOXEIH3': ['similar',
  'old',
  'last',
  'much',
  'next',
  'good',
  'rescu',
  'much',
  'equip',
  'various',
  'direct',
  'explan',
  'direct',
  'otherwis',
  'dry',
  'cheap'],
 'R3VHL6UVB9OO6Q': ['give', 'fair', 'low', 'big', 'someth', 'big', 'good'],
 'RWEXSA2MD55OL': ['new',
  'dry',
  'buttonscontrol',
  'good',
  'dry',
  'conair',
  'glad',
  'new',
  'anoth'],
 'R28QOD06DS9LXZ': ['great'],
 'R1QRA8W0AQHK6U': ['dry', 'perfect', 'satisfi'],
 'RNM72XACDNTQ1': ['great', 'dont'],
 'R3GVXOMJFFJ1D1': ['good'],
 'R2HMVWO6I8HBCC': ['littl',
  'dry',
  'trick',
  'thick',
  'thin',
  'super',
  'hot',
  'good',
  'damag',
  'dry',
  'own',
  'last',
  'happi',
  'small'],
 'RYNBIV19CNV2N': ['great'],
 'R3JLOSCFGXH97A': ['gr

In [10]:
a = Apriori(min_sup=0.025,dataDic = df1)
L = a.do()
print(L)

[['nice'], ['high'], ['fine'], ['good'], ['thick'], ['dry'], ['old'], ['last'], ['much'], ['low'], ['big'], ['new'], ['great'], ['perfect'], ['littl'], ['hot'], ['own'], ['small'], ['ive'], ['strong'], ['quiet'], ['worth'], ['hair'], ['cool'], ['dri'], ['quick'], ['light'], ['soft'], ['actual'], ['long'], ['compact'], ['dry', 'thick'], ['dry', 'good'], ['dry', 'old'], ['dry', 'last'], ['dry', 'much'], ['dry', 'new'], ['dry', 'hot'], ['dry', 'small'], ['good', 'great'], ['dry', 'quiet'], ['dri', 'great'], ['dry', 'great'], ['dry', 'ive'], ['dry', 'quick'], ['dry', 'high'], ['dry', 'hair'], ['dri', 'dry'], ['dry', 'nice'], ['dry', 'low'], ['great', 'high'], ['dry', 'long'], ['dry', 'good', 'great']]
